In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import regionmask

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import tensorflow as tf

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Step 1: Renewable Energy data

- **Source:** https://data.open-power-system-data.org/time_series/2019-06-05


- **Description:** This data package contains different kinds of timeseries data relevant for power system modelling, namely electricity consumption (load) for 37 European countries as well as wind and solar power generation and capacities and prices for a growing subset of countries. The timeseries become available at different points in time depending on the sources. The data has been downloaded from the sources, resampled and merged in a large CSV file with hourly resolution. Additionally, the data available at a higher resolution (Some renewables in-feed, 15 minutes) is provided in a separate file. All data processing is conducted in python and pandas and has been documented in the Jupyter notebooks linked below.


#### This dataset is a consolidation from several different sources. Due to different quality standards for different countries, for the example in the current notebook let's use data for Germany (country code = 'DE_')


In [ ]:
# energy data file
energy_fn = './data/energy/time_series_60min_singleindex.csv.gz'

# country to be extracted from the energy dataset
country_code = 'DE_'

# function to extract only the needed columns
usecols = lambda colname: colname.startswith('utc') | colname.startswith(country_code)

# reading CSV into a pd.DataFrame, selecting columns with 'usecols'
energy_df = pd.read_csv(energy_fn,
                        usecols=usecols,          
                        parse_dates=['utc_timestamp'],
                        index_col=['utc_timestamp'])

# slicing a period of the data
energy_df = energy_df['2016-01-01 00:00:00':'2018-12-31 23:00:00']
energy_df.head()

In [ ]:
# barplots are slow when many vertical bars have to be drawn, be patient (or change to lineplot)
plt.figure(figsize=(18,8))
plt.bar(energy_df.index, energy_df[f'{country_code}solar_generation_actual'], color='#f01212')
## use this line if impatient
# plt.plot(energy_df.index, energy_df.NL_solar_generation_actual)
plt.xticks(rotation=60)
plt.title(f'Actual solar generation for {country_code[:-1]}')
plt.ylabel('MW')
plt.xlabel('hourly data')
plt.show()

In [ ]:
# barplots are slow when many vertical bars have to be drawn, be patient (or change to lineplot)
plt.figure(figsize=(18,8))
plt.bar(energy_df.index, energy_df[f'{country_code}wind_offshore_generation_actual'], color='#196DAE')
## use this line if impatient
# plt.plot(energy_df.index, energy_df.NL_wind_offshore_generation_actual)
plt.xticks(rotation=60)
plt.title(f'Actual wind generation for {country_code[:-1]}')
plt.ylabel('MW')
plt.xlabel('hourly data')
plt.show()

# Step 2: Meteorological data 

- Let's use pre-downloaded hourly data, link provided in README
- Hourly data gives us the same time granularity as in the energy data. We'll slice the data accordingly so both datasets cover the exact same period (and have the same shape)
- Meteorological features for prediction are:

|Variable|Variable Name|
|-------:|------------:|
|t2m|2 metre temperature|
|msl|Mean sea level pressure|
|u10|10 metre U wind component|
|v10|10 metre V wind component|
|u100|100 metre U wind component|
|v100|100 metre V wind component|
|fsr|Forecast surface roughness|
|cdir|Clear-sky direct solar radiation at surface|
|ssrdc|Surface solar radiation downward clear-sky|
|ssrd|Surface solar radiation downwards|
|tisr|TOA incident solar radiation|


In [ ]:
# load wx data for each year
wx_data_2016 = xr.open_dataset('./data/weather/era5_DE_hourly_2016.nc')
wx_data_2017 = xr.open_dataset('./data/weather/era5_DE_hourly_2017.nc')
wx_data_2018 = xr.open_dataset('./data/weather/era5_DE_hourly_2018.nc')

# combining wx data, cleaning unused object for memory efficiency
wx_data = xr.concat([wx_data_2016, wx_data_2017, wx_data_2018], dim='time')
del wx_data_2016, wx_data_2017, wx_data_2018

In [ ]:
# let's re-use the extract function from previous example
country_mask = regionmask.defined_regions.natural_earth.countries_50.mask(wx_data, 
                                                                          lon_name='longitude', 
                                                                          lat_name='latitude')

def extract_data_for_country(country_name, country_mask, wx_data):
    country_id = regionmask.defined_regions.natural_earth.countries_50.map_keys(country_name)
    wx_data = wx_data.where(country_mask==country_id)
    wx_data = wx_data.dropna('latitude', how='all')
    wx_data = wx_data.dropna('longitude', how='all')
    return wx_data

In [ ]:
wx_data = extract_data_for_country('Germany', country_mask, wx_data)

In [ ]:
plt.clf()
data_crs = ccrs.PlateCarree()

plt.figure(figsize=(13,13))
data_crs = ccrs.PlateCarree()

ax = plt.axes(projection=data_crs)
wx_data.t2m.sel(time='2016-06-01 00:00:00').plot(ax=ax,transform=data_crs, cmap='viridis')
plt.show()

# Step 3a: Modeling for Wind Energy

- How do the same points from the previous example work in a neural network case?

## Important considerations:
    - Training/*Validation*/Test Split
    - Features to be used
    - Algorithm to choose
        - # layers
        - # units
        - Neural networks hyperparameters
    - Always check your data

In [ ]:
train_period = ('2016-01-01 00:00:00','2017-12-31 23:00:00')
valid_period = ('2018-01-01 00:00:00','2018-06-30 23:00:00')
test_period  = ('2018-07-01 00:00:00','2018-12-31 23:00:00')

### Neural Network explanation

In [ ]:
# slicing train, valid and test data
features = ['t2m', 'msl', 'u10', 'v10', 'u100', 'v100']

X_train = wx_data[features].sel(time=slice(train_period[0],train_period[1])).mean(axis=(1,2)).to_dataframe()
X_valid = wx_data[features].sel(time=slice(valid_period[0],valid_period[1])).mean(axis=(1,2)).to_dataframe()
X_test = wx_data[features].sel(time=slice(test_period[0],test_period[1])).mean(axis=(1,2)).to_dataframe()

# saving column names for later
cols = X_train.columns

In [ ]:
X_train.head()

# Why do we scale the data before ingesting in NNs?
![](./imgs/scaling.png)

In [ ]:
# scaling X data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.values)
X_valid = scaler.transform(X_valid.values)
X_test = scaler.transform(X_test.values)

X_train = pd.DataFrame(data=X_train, columns=cols)
X_train.index.names = ['time']

X_valid = pd.DataFrame(data=X_valid, columns=cols)
X_valid.index.names = ['time']

X_test = pd.DataFrame(data=X_test, columns=cols)
X_test.index.names = ['time']

In [ ]:
X_train.head()

In [ ]:
X_train.shape, X_valid.shape, X_test.shape 

In [ ]:
# Slicing Train, Test and Validation Sets
labels = [f'{country_code}wind_onshore_generation_actual']
Y_train = energy_df[labels][train_period[0]:train_period[1]]
Y_valid = energy_df[labels][valid_period[0]:valid_period[1]]
Y_test = energy_df[labels][test_period[0]:test_period[1]]

Y_train.shape, Y_valid.shape, Y_test.shape

In [ ]:
# filtering some NaN's present in labels

X_train = X_train[Y_train.notnull().values]
Y_train = Y_train[Y_train.notnull().values]

X_valid = X_valid[Y_valid.notnull().values]
Y_valid = Y_valid[Y_valid.notnull().values]

X_test = X_test[Y_test.notnull().values]
Y_test = Y_test[Y_test.notnull().values]

X_train.shape, X_valid.shape, X_test.shape,Y_train.shape, Y_valid.shape, Y_test.shape

# Some comments on Keras module


- Documentation: https://www.tensorflow.org/api_docs/python/tf/keras


#### Activation functions: https://www.tensorflow.org/api_docs/python/tf/keras/activations
- Let's use the 'rectified linear unit' (relu) activation function

#### Optimizers:
- Let's use the Adam optmizer: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
    - Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments. According to the paper [Adam: A Method for Stochastic Optimization. Kingma et al., 2014,](http://arxiv.org/abs/1412.6980) the method is "computationally efficient, has little memory requirement, invariant to diagonal rescaling of gradients, and is well suited for problems that are large in terms of data/parameters".
    
- we won't change the default parameters of the Adam optmizer:
```
    __init__(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name='Adam',
        **kwargs
        )
```

#### Initializers: https://www.tensorflow.org/api_docs/python/tf/keras/initializers
- Let's use a RandomNormal() initalizer


### Why do we have to define `Checkpoint` and `Earlystop`?


In [ ]:
activation = tf.keras.activations.relu
optimizer = tf.keras.optimizers.Adam()
initializer = tf.keras.initializers.RandomNormal(dtype=tf.dtypes.float32)
loss = tf.keras.losses.MeanAbsoluteError()

num_hidden_layers = 10
num_units = 50

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(num_units, 
                                kernel_initializer=initializer,
                                input_dim=X_train.shape[1]))

for layers in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(num_units, 
                                    kernel_initializer=initializer,
                                    activation=activation))
    
model.add(tf.keras.layers.Dense(1, 
                                kernel_initializer=initializer,
                                activation=activation))

model.compile(loss=loss,
              optimizer=optimizer,
              metrics=[loss])

print(model.summary())

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model_wind.h5',
                                                monitor="val_loss",
                                                verbose=0,
                                                save_weights_only=True,
                                                mode="auto",
                                                save_freq=1)

earlystop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                             min_delta=0, 
                                             patience=10, 
                                             verbose=0, 
                                             mode="auto")

history = model.fit(X_train.values, 
                    Y_train.values.reshape(-1),
                    validation_data=(X_valid, Y_valid),
                    epochs = 100, 
                    batch_size=512,
                    callbacks=[checkpoint, earlystop])

### Before testing the model, let's analyze what happened during training

In [ ]:
history.history.keys()
plt.figure(figsize=(12,8))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Training')
plt.xlabel('epoch number')

plt.axvline(np.argmin(history.history['val_loss']), 
            linestyle='--',
            linewidth=1)

plt.axhline(np.min(history.history['val_loss']), 
            linestyle='--',
            linewidth=1,
            label='best model')

plt.legend()
plt.show()

### Ok now let's predict

In [ ]:
model.load_weights('best_model_wind.h5')
Y_pred = model.predict(X_test)

In [ ]:
mae = metrics.mean_absolute_error(Y_test, Y_pred)
mse = metrics.mean_squared_error(Y_test, Y_pred)
msle = metrics.mean_squared_log_error(Y_test, Y_pred)
r2 = metrics.r2_score(Y_test, Y_pred)

In [ ]:
print(f"Mean Absolute Error - MAE = {mae}")
print(f"Mean Squared Error - MSE = {mse}")
print(f"Mean Squared Log Error - MSLE = {msle}")
print(f"R**2 Score - R2 = {r2}")


In [ ]:
plt.style.use('fivethirtyeight')
plt.figure(figsize=(16,8))
plt.plot(Y_test.values, label='Observations')
plt.plot(Y_pred, label='Prediction')
plt.legend()
plt.title('Model Performance for Wind Energy Generation')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(Y_test, Y_pred)
l_min, l_max = -100,  np.max([np.max(Y_test.values),np.max(Y_pred)])+200
plt.xlim([l_min, l_max])
plt.ylim([l_min, l_max])
plt.plot([l_min, l_max],[l_min, l_max], color='k', lw=1)
plt.title('Scatterplot - Wind Model Performance')
plt.xlabel('Observsations')
plt.ylabel('Predictions')
plt.tight_layout()
plt.show()

# Step 3b: Modeling for Solar Energy

- Let's use different meteorological features now:

In [ ]:
# slicing train, valid and test data
features = ['t2m', 'cdir', 'ssrdc', 'ssrd', 'tisr', 'msl'] 

X_train = wx_data[features].sel(time=slice(train_period[0],train_period[1])).mean(axis=(1,2)).to_dataframe()
X_valid = wx_data[features].sel(time=slice(valid_period[0],valid_period[1])).mean(axis=(1,2)).to_dataframe()
X_test = wx_data[features].sel(time=slice(test_period[0],test_period[1])).mean(axis=(1,2)).to_dataframe()

# saving column names for later
cols = X_train.columns

In [ ]:
X_train.head()

In [ ]:
# scaling X data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.values)
X_valid = scaler.transform(X_valid.values)
X_test = scaler.transform(X_test.values)

X_train = pd.DataFrame(data=X_train, columns=cols)
X_train.index.names = ['time']

X_valid = pd.DataFrame(data=X_valid, columns=cols)
X_valid.index.names = ['time']

X_test = pd.DataFrame(data=X_test, columns=cols)
X_test.index.names = ['time']

In [ ]:
X_train.tail()

In [ ]:
# Slicing Train, Test and Validation Sets
labels = [f'{country_code}solar_generation_actual']
Y_train = energy_df[labels][train_period[0]:train_period[1]]
Y_valid = energy_df[labels][valid_period[0]:valid_period[1]]
Y_test = energy_df[labels][test_period[0]:test_period[1]]

Y_train.shape, Y_valid.shape, Y_test.shape

In [ ]:
# filtering some NaN's present in labels

X_train = X_train[Y_train.notnull().values]
Y_train = Y_train[Y_train.notnull().values]

X_valid = X_valid[Y_valid.notnull().values]
Y_valid = Y_valid[Y_valid.notnull().values]

X_test = X_test[Y_test.notnull().values]
Y_test = Y_test[Y_test.notnull().values]

X_train.shape, X_valid.shape, X_test.shape,Y_train.shape, Y_valid.shape, Y_test.shape

In [ ]:
activation = tf.keras.activations.relu
optimizer = tf.keras.optimizers.Adam()
initializer = tf.keras.initializers.RandomNormal(dtype=tf.dtypes.float32)
loss = tf.keras.losses.MeanAbsoluteError()

num_hidden_layers = 10
num_units = 80

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(num_units, 
                                kernel_initializer=initializer,
                                input_dim=X_train.shape[1]))

for layers in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(num_units, 
                                    kernel_initializer=initializer,
                                    activation=activation))
    
model.add(tf.keras.layers.Dense(1, 
                                kernel_initializer=initializer,
                                activation=activation))

model.compile(loss=loss,
              optimizer=optimizer,
              metrics=[loss])

print(model.summary())

checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model_solar.h5',
                                                monitor="val_loss",
                                                verbose=0,
                                                save_weights_only=True,
                                                mode="auto",
                                                save_freq=1)

earlystop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                             min_delta=0, 
                                             patience=10, 
                                             verbose=0, 
                                             mode="auto")

history = model.fit(X_train.values, 
                    Y_train.values.reshape(-1),
                    validation_data=(X_valid, Y_valid),
                    epochs = 100, 
                    batch_size=512,
                    callbacks=[checkpoint, earlystop])

In [ ]:
model.load_weights('best_model_solar.h5')
Y_pred = model.predict(X_test)
mae = metrics.mean_absolute_error(Y_test, Y_pred)
mse = metrics.mean_squared_error(Y_test, Y_pred)
msle = metrics.mean_squared_log_error(Y_test, Y_pred)
r2 = metrics.r2_score(Y_test, Y_pred)
print(f"Mean Absolute Error - MAE = {mae}")
print(f"Mean Squared Error - MSE = {mse}")
print(f"Mean Squared Log Error - MSLE = {msle}")
print(f"R**2 Score - R2 = {r2}")

In [ ]:
plt.style.use('fivethirtyeight')
plt.figure(figsize=(16,8))
plt.plot(Y_test.values, label='Observations')
plt.plot(Y_pred, label='Prediction', alpha=0.65)
plt.legend()
plt.title('Model Performance for Solar Energy Generation')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(Y_test, Y_pred)
l_min, l_max = -600,  np.max([np.max(Y_test.values),np.max(Y_pred)])+200
plt.xlim(l_min, l_max)
plt.ylim(l_min, l_max)
plt.plot([l_min, l_max],[l_min, l_max], color='k', lw=1)
plt.title('Scatterplot - Solar Model Performance')
plt.xlabel('Observsations')
plt.ylabel('Predictions')
plt.tight_layout()
plt.show()